In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 池化层前向传播的实现

In [2]:
# 此实现不支持修改padding和stride
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    # 初始化输出
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max() # 取窗口内元素最大值
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean() # 取窗口内元素均值
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], 
                  [3.0, 4.0, 5.0], 
                  [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## 池化层中的填充与步幅
和以前一样，我们可以通过填充和步幅以获得所需的输出形状

In [13]:
# 构造一个输入张量X
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4)) # batch_size, channel, h, w
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同

In [6]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

我们可以设定一个任意大小的窗口，并分别设定填充和步幅的高度和宽度

In [7]:
pool2d = nn.MaxPool2d((2, 3), # 窗口大小 2 x 3
                      stride=(2, 3), # 高（垂直方向）步幅=2，宽（水平方向）步幅=3
                      padding=(0, 1)) # 高两边填充0行，宽左右各填充1列
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 多个通道
池化层的输出通道=输入通道

In [14]:
X = torch.cat((X, X + 1), 1) # 在通道维度上连接张量
X  # X.shape=(1, 2, 4, 4)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [15]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X) # 输出通道依然是2

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

池化的主要优点之一是减轻卷积层对位置的过度敏感